In [1]:
import matplotlib.pyplot as plt
import numpy as np

from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

from sklearn.model_selection import train_test_split

seed = 3244
np.random.seed(seed)

h = 50  # height of image
w = 37  # width of image
num_pixels = h * w
num_classes = 7

Using TensorFlow backend.


In [2]:
X = np.load('X_train.npy') # (996, 1850)
y = np.load('y_train.npy') # (996, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

In [3]:
def shapeData(data, h=50, w=37):
    return data.reshape(data.shape[0], 1, h, w).astype('float32')

X_train = shapeData(X_train)
X_test = shapeData(X_test)

In [4]:
def normalize(data):
    return data / 255

X_train = normalize(X_train)
X_test = normalize(X_test)

In [5]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [6]:
def baseline_model():
    model = Sequential()
    model.add(Convolution2D(32, 5, 5, border_mode='valid', input_shape=(1, h, w), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [7]:
num_filters = 32  # number of convolutional filters to use
pool_size = (2, 2)  # size of pooling area for max pooling
kernel_size = (5, 5)  # convolution kernel size

def model1():
    model = Sequential()
    model.add(Convolution2D(30, 5, 5, border_mode='valid', input_shape=(1, h, w), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Convolution2D(15, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def model2():
    model = Sequential()
    model.add(Convolution2D(32, 5, 5, border_mode='valid', input_shape=(1, h, w), activation='relu'))
    model.add(Convolution2D(32, 5, 5, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Convolution2D(64, 3, 3, border_mode='valid', activation='relu'))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    # Note: Keras does automatic shape inference.
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(num_classes, activation='softmax'))
    
    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

def model3():
    model = Sequential()
    model.add(Convolution2D(num_filters, kernel_size[0], kernel_size[1],
                            border_mode='valid',
                            input_shape=(1, h, w)))
    model.add(Activation('relu'))
    model.add(Convolution2D(num_filters, kernel_size[0], kernel_size[1]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])
    return model

In [8]:
model = baseline_model()
# model = model1()
# model = model2()
# model = model3()

nb_epoch = 40
batch_size = 50

model.fit(X_train, y_train, 
          validation_data=(X_test, y_test), 
          nb_epoch=nb_epoch, 
          batch_size=batch_size, 
          verbose=1)

scores = model.evaluate(X_test, y_test, verbose=0)
print("Model accuracy: %.2f%%" % (scores[1]*100))

Train on 772 samples, validate on 194 samples
Epoch 1/40
772/772 [==============================] - 4s - loss: 1.7833 - acc: 0.3368 - val_loss: 1.6628 - val_acc: 0.4278
Epoch 2/40
772/772 [==============================] - 4s - loss: 1.6582 - acc: 0.4080 - val_loss: 1.5848 - val_acc: 0.4278
Epoch 3/40
772/772 [==============================] - 4s - loss: 1.5709 - acc: 0.4262 - val_loss: 1.5752 - val_acc: 0.5773
Epoch 4/40
772/772 [==============================] - 4s - loss: 1.4563 - acc: 0.4961 - val_loss: 1.3636 - val_acc: 0.4794
Epoch 5/40
772/772 [==============================] - 4s - loss: 1.2817 - acc: 0.5415 - val_loss: 1.1947 - val_acc: 0.5928
Epoch 6/40
772/772 [==============================] - 4s - loss: 1.0957 - acc: 0.6244 - val_loss: 1.0734 - val_acc: 0.6134
Epoch 7/40
772/772 [==============================] - 4s - loss: 0.9047 - acc: 0.7060 - val_loss: 0.9217 - val_acc: 0.6392
Epoch 8/40
772/772 [==============================] - 6s - loss: 0.7526 - acc: 0.7630 - val_l